# Chapter 15 - Multiple Regression

Introducing more variables. Beta in this case is not a single variable but a vector of variables, each representing the relationship of that variable to the prediction

In [12]:
# imports and functions from other chapters. Someday I'll refactor and make importable packages out of all the old functions
# but for now this is it
import random

def predict(alpha, beta, x_i):
    return beta * x_i + alpha

def dot(v, w):
    return sum(v_i * w_i
              for v_i, w_i in zip(v,w))

def median(v):
    n = len(v)
    sorted_v = sorted(v)
    midpoint = n//2
    
    if n % 2 == 1:
        return sorted_v[midpoint]
    else:
        lo = midpoint -1
        hi = midpoint # I want to get to the point where I intuit whether to add one to get hi or subtract one to get lo
                      # at 10 pm on a thursday some day
        return (sorted_v[lo] + sorted_v[hi]) / 2

def mean(x):
    return sum(x) / len(x)

def de_mean(x):
    x_bar = mean(x)
    return [x_i - x_bar for x_i in x]

def total_sum_of_squares(y):
    return sum(v ** 2 for v in de_mean(y))

def in_random_order(data):
    indices = [i for i, _ in enumerate(data)]
    random.shuffle(indices) #Joel writes indexes but that's not the pluralization.
    for i in indices:
        yield data[i]

def minimize_stochastic(target_fn, gradient_fn, x, y, theta_0, alpha_0=0.01):
    
    data = zip(x,y,)
    theta = theta_0
    alpha = alpha_0
    
    min_theta, min_value = None, float('inf')
    iterations_with_no_improvement = 0
    
    # we want to stop when we don't see any improvement try shrinking the step size, but if we do keep going
    while iterations_with_no_improvement < 100:
        value = sum( target_fn(x_i, y_i, theta) for x_i, y_i in data )
        
        if value < min_value:
            min_theta, min_value = theta, value
            iterations_with_no_improvement = 0
            alpha = alpha_0
        else:
            iterations_with_no_improvement += 1
            alpha *= 0.9
            
        # take a gradient step
        for x_i, y_i in in_random_order(data):
            gradient_i = gradient_fn(x_i, y_i, that)
            theta = vector_subtract(theta, scalar_multiply(alpha, gradient_i))
            
    return min_theta


In [4]:
# beta = [alpha, beta_1, ..., beta_k]
# x_i = [1, x_i1, ..., x_ik]

#new predict function
def predict(x_i, beta):
    return dot(x_i, beta)

x = [1, 49, 4, 0] # independent variable with constant term, num of friends, work hours per day, and boolean "has a Phd"

Important assumptions of the least squares model is that each of the columns of x are linearly independent, meaning there is no way to write any parameter as a weighted sum of the others. Violations of this assumption are typically nonobvious 

Another is that the parameters of x are all uncorrelated with the errors. Otherwise our estimate of beta will be systematically wrong.

In [6]:
# Similar to linear regression from previous chapter: minimize error function with stochastic gradient descent

def error(x_i, y_i, beta):
    return y_i - predict(x_i, beta)

def squared_errors(x_i, y_i, beta):
    return error(x_i, y_i, beta) ** 2

def squared_error_gradient(x_i, y_i, beta): # hooray calculus
    return [-2 * x_ij * error(x_i, y_i, beta) for x_ij in x_i]

# now we can find beta using SGD
def estimate_beta(x,y):
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(squared_error, squared_error_gradient, x,y, beta_initial, 0.001)

In [8]:
# example

random.seed(0)
beta = estimate_beta(x, daily_minutes_good)
# output = [30.62, 0.972, -1.868, 0.911]

NameError: name 'daily_minutes_good' is not defined

This means that our model looks like:
minutes = 30.63 + 0.972*friends - 1.868*work hours + 0.911*has PhD

Worth reading the chapter here because he starts to allow for interactions among the variables, and suggests handling cases for multiplying the variables, etc. This is obviously leading to CNN type deep learing stuff that will just do that on its own (account for interactions among variables, that is). He ends by asking if these coefficients matter because there are no limits to the numbers of products, logs, squares, and higher powers we could add.

In [9]:
# goodness of fit

def multiple_r_squared(x,y,beta):
    sum_of_squared_errors = sum(error(x_i, y_i, beta) ** 2 for x_i, y_i in zip (x,y))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(y)

# we also want to know the errors on each individual coefficient and whether they mean anything, but we are not
# set up to do that from scratch

## Digression: The Bootstrap

Imagine a sample of n data point generated by some distribution:

data = get_sample(num_points=n)

We can get the median of the sample, but it varies considerably depending on the data (100 points near 100 or 100 points nearly equally distributed near 0 and near 200). Usually we can't get new samples and keep computing the median of each. What we can do instead is bootstrap (woo) new data sets by n data points with replacement from our data and then compute the medians from those synthetic datasets.

In [19]:
def bootstrap_sample(data):
    return [random.choice(data) for _ in data]

def bootstrap_statistic(data, stats_fn, num_samples):
    return [stats_fn(bootstrap_sample(data)) for _ in range(num_samples)]

In [22]:
# for example, these data sets

close_to_100 = [99.5 + random.random() for _ in range(101)]
far_from_100 = ([99.5 + random.random()] + [random.random() for _ in range(50)] + [200 + random.random() for _ in range(50)])

print("Median of each set: ", median(close_to_100),",", median(far_from_100))

bootstrap_statistic(close_to_100, median, 100)
bootstrap_statistic(far_from_100, median, 100)


Median of each set:  99.94405543380051 , 100.04454803230855


[200.11259534348648,
 200.09491321652237,
 200.02359714875064,
 200.08067194244424,
 0.9524031774659693,
 0.9524031774659693,
 200.099964346878,
 200.08067194244424,
 200.02122396547387,
 200.1015980133037,
 200.09491321652237,
 200.11259534348648,
 0.9524031774659693,
 200.09491321652237,
 200.08067194244424,
 200.099964346878,
 200.1043347115292,
 0.9980053090153341,
 0.94750489984989,
 100.04454803230855,
 200.08067194244424,
 200.02359714875064,
 0.7502700517993686,
 0.6758674507055095,
 0.8081023910411597,
 200.08067194244424,
 200.02359714875064,
 0.9980053090153341,
 0.9524031774659693,
 200.02359714875064,
 200.17030169808345,
 200.09491321652237,
 0.8366818418224948,
 200.02359714875064,
 0.9524031774659693,
 200.11259534348648,
 200.099964346878,
 200.02122396547387,
 200.02359714875064,
 200.099964346878,
 0.761559089380398,
 200.1015980133037,
 0.7502700517993686,
 200.02122396547387,
 100.04454803230855,
 100.04454803230855,
 200.02122396547387,
 0.9524031774659693,
 0.947

In [23]:
# we can take the same approach to estimating the errors in our regression coefficients

def estimate_sample_beta(sample):
    x_sample, y_sample = zip(*sample) # magic unzipping trick
    return estimate_beta(x_sample, y_sample)

In [24]:
# example using chapter 5 stuff

random.seed(0)
bootstrap_betas = bootstrap_statistic(zip(x, daily_minutes_good), estimate_sample_beta, 100)

bootstrap_standard_errors = [standard_deviation([beta[i] for beta in bootstrap_betas]) for i in range(4)]


NameError: name 'daily_minutes_good' is not defined

In [26]:
# we can test each beta against the null hypothesis with a student's t distribution (n-k DOF) or since n is much larger
# than k, a normal_cdf

def p_value(beta_hat_j, sigma_hat_j):
    if beta_hat_j > 0:
        return 2 * (1-normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j)
    
p_value(30.63, 1.174) # constant term, ~0
p_value(0.972, 0.079) # num_friends, ~0
p_value(-1.868, 0.131) # work_hours, ~0
p_value(0.911, 0.990) # PhD, 0.36

# if we weren't going from scratch we could compute the t-distrition and the exact standard errors
# Most of the coefficients have very small p-values except PhD, meaning it is probably not meaningful


NameError: name 'normal_cdf' is not defined

## Regularization

If we want to actually explain phenomena, it doesn't help if we have hundreds of coefficients even if the model is really good

We can add a penalty to the error term that gets larger as beta gets larger, valuing less coefficients, and minimize the combined error and penalty.

In [27]:
def ridge_penalty(beta, alpha):
    return alpha * dot(beta[1:], beta[1:])

def squared_error_ridge(x_i, y_i, beta, alpha):
    return error(x_i, y_i, beta) ** 2 + ridge_penalty(beta, alpha)

# which we can then plug into SGD in the usual way

def ridge_penalty_gradient(beta, alpha):
    return [0] + [2 * alpha * beta_j for beta_j in beta[1:]]

def squared_error_ridge_gradient(x_i, y_i, beta, alpha):
    return vector_add(squared_error_gradient(x_i, y_i, beta), ridge_penalty_gradient(beta, alpha))

def estimate_beta_ridge(x,y,alpha):
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(partial(squared_error_ridge, alpha=alpha), partial(squared_error_ridge_gradient, alpha=alpha), 
                              x, y, beta_initial, 0.001)

# as we increase alpha, the goodness of fit gets worse, but the size of beta gets smaller. See book for actual values
# the coefficient on PhD vanishes as we increase the penalty

#Note: typically we want to rescale our data before using this approach. If we changed years experience to
#centuries, it's coefficient would increase by a factor of 100 and get penalized much more even though it's the same model

In [28]:
# Another approach is to use lasso penalty regression

def lasso_penalty(beta, alpha):
    return alpha * sum(abs(beta_i) for beta_i in beta[1:])


Whereas the ridge penalty shrinks coefficients overall, lasso tends to force them to be zero which is good for learing sparse models. However, it is not amenable to gradient descent which means we won't be able to solve it from scratch

## This concludes Chapter 15

For further exploration read wikipedia pages on Regression. It has a rich history. Scikit-learn has a linear_model module that provides a linear regression model similar to this one as well as many types of regularization. Statsmodels is another Python module that has linear regression models.